In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.metrics import * 
from sklearn import preprocessing

2024-09-11 16:08:38.659917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
label_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

In [4]:
## Importing Data ##

# Get training data
train_df = pd.read_csv('fashion-mnist_train.csv')
train_df.loc[:, 'label'] = train_df.loc[:, 'label'].map(label_mapping)
x_train = train_df.iloc[:,1:]
y_train = train_df.loc[:, 'label']

In [5]:
## Data Preprocessing ##

#Training
encoder = preprocessing.OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(train_df.loc[:,'label'].values.reshape(-1,1)) # OneHot encoding
feature_names = [name.split('_', 1)[1] for name in encoder.get_feature_names_out()]

x_train/=255  # Normalize
y_train = pd.DataFrame(y_train, columns=feature_names) 

In [9]:
## Defining a Callback ##

class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.95:                 
                print("\nReached 95% accuracy so cancelling training!")
                
                # Stop training once the above condition is met
                self.model.stop_training = True

In [10]:
## Model Creation and Training ##

def train_model(x_train, y_train):

    
    # Instantiate the callback class
    callbacks = myCallback()
    
    # Define the model
    model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                        tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        
        
    ]) 
    
    # Compile the model
    model.compile(optimizer='adam',                   
                  loss='categorical_crossentropy',                   
                  metrics=['accuracy'])     
    
    # Fit the model for 10 epochs adding the callbacks
    # and save the training history
    history = model.fit(x_train, y_train, epochs=50, callbacks=[callbacks])

    ### END CODE HERE

    return model

model = train_model(x_train, y_train)

Epoch 1/50
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4807 - accuracy: 0.8283
Epoch 2/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3639 - accuracy: 0.8672
Epoch 3/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3267 - accuracy: 0.8792
Epoch 4/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3022 - accuracy: 0.8880
Epoch 5/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2832 - accuracy: 0.8954
Epoch 6/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2703 - accuracy: 0.9000
Epoch 7/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2558 - accuracy: 0.9048
Epoch 8/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2458 - accuracy: 0.9086
Epoch 9/50
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2341 - accuracy: 0.9114
Epoch 10/50
1875/1875 [==============================] - 7s 4ms/step - loss: 0.227

In [14]:
model.save('clothing-classifier-model.h5')

In [13]:
import matplotlib.pyplot as plt

# Assuming your array is named 'pixel_array' and has length 28^2 = 784
pixel_array = np.array(x_train.iloc[20,:]) # Example array with values between 0-255


model.predict(pixel_array)
# Reshape the array into a 28x28 matrix
image = pixel_array.reshape(28, 28)

# Display the image
plt.imshow(image, cmap='gray')
plt.axis('off')  # Turn off the axis
plt.show()

print(pixel_array)


ValueError: in user code:

    File "/Users/jonathanvarghese/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/Users/jonathanvarghese/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/jonathanvarghese/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/jonathanvarghese/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/Users/jonathanvarghese/anaconda3/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/jonathanvarghese/anaconda3/lib/python3.10/site-pack